In [1]:
# Most importantly, select the kernal at least 3.11

In [1]:
import os, subprocess
import torch, struct, os, psutil, subprocess, time, threading
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import tensorflow as tf

from torch.utils.data import DataLoader, TensorDataset

2024-02-04 21:17:21.507285: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-04 21:17:21.626472: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-04 21:17:21.626533: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-04 21:17:21.639000: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-04 21:17:21.676958: I tensorflow/core/platform/cpu_feature_guar

## Prepare

In [2]:
# Load TensorFlow MNIST data
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Convert to PyTorch tensors
test_images_pt = torch.tensor(test_images).float()
test_labels_pt = torch.tensor(test_labels)
# Flatten and normalize the images
test_images_pt = test_images_pt.view(-1, 28*28) / 255.0  # Flatten and normalize

# Assuming test_images_pt is your PyTorch tensor with shape [num_samples, 784]
test_images_pt_reshaped = test_images_pt.view(-1, 1, 28, 28)  # Reshape to [num_samples, channels, height, width]

# Downsample images
test_images_pt_downsampled = F.interpolate(test_images_pt_reshaped, size=(14, 14), mode='bilinear', align_corners=False)

# Flatten the images back to [num_samples, 14*14]
test_images_pt_downsampled = test_images_pt_downsampled.view(-1, 14*14)

In [3]:
def evaluate_pytorch_model(model, datasets, labels):
    # Create TensorDataset for test data
    test_dataset = TensorDataset(datasets, labels)
    # Create a DataLoader for the test dataset
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy

## Benchmark for 784_56_10 DNN Model

### Conver ONNX model to cairo

In [9]:
# After login
arch_folder = "input-dense-dense/"
layers = [784,56,10]
model_name = "_".join([str(x) for x in layers])
model_path = "../../models/" + arch_folder + model_name

state_dict = torch.load(model_path + ".pth")

output_folder = './tmp/' + "_".join([str(x) for x in layers]) + "/"
os.makedirs(output_folder, exist_ok=True)


In [11]:
class Net(nn.Module):
    def __init__(self, num_classes=10):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(layers[0], layers[1])  # Flatten 
        self.fc2 = nn.Linear(layers[1], layers[2])  

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
model_pt = Net()
model_pt.load_state_dict(state_dict)
model_pt.eval()  # Set the model to evaluation mode

with torch.no_grad():  # Ensure gradients are not computed
    predictions = model_pt(test_images_pt)
    predicted_labels = predictions.argmax(dim=1)

predicted_labels = predicted_labels.tolist()

In [13]:
# Evaluate the PyTorch model
accuracy = evaluate_pytorch_model(model_pt, test_images_pt, test_labels_pt)
print(f'Accuracy of the PyTorch model on the test images: {accuracy:.8f}%')

Accuracy of the PyTorch model on the test images: 97.40000000%


#### Save the Model locally (no softmax as not supported)

In [15]:
torch.onnx.export(model_pt, test_images_pt[0], "784_mnist.onnx")

In [21]:
command = ['giza', 'transpile', "784_mnist.onnx", '--output-path', output_folder]

In [22]:
subprocess.run(command)

[giza][2024-02-04 21:27:24.815] No model id provided, checking if model exists ✅
[giza][2024-02-04 21:27:24.817] Model name is: 784_mnist
[giza][2024-02-04 21:27:25.267] Model Created with id -> 259! ✅
[giza][2024-02-04 21:27:26.004] Sending model for transpilation ✅ 
⠹ Transpiling Model...
[giza][2024-02-04 21:27:45.010] Transpilation recieved! ✅
[giza][2024-02-04 21:27:45.015] Transpilation saved at: ./tmp/784_56_10/


CompletedProcess(args=['giza', 'transpile', '784_mnist.onnx', '--output-path', './tmp/784_56_10/'], returncode=0)

### Deploy ml service

In [ ]:
sierra_path = output_folder + "inference/" 
command = ['giza', 'deployments', 'deploy', ]

### Create input.cairo

In [8]:
input_path = output_folder + "inputs/"
os.makedirs(input_path, exist_ok=True)

In [9]:
def to_fixed_point(val, bits):
    return round(val * (2**bits))

def mnist_image_to_fixed_point(data):
    return [to_fixed_point(val.item(), 16) for val in data]


def generate_input_cairo(data):
    values = mnist_image_to_fixed_point(data)
    values = [f"FixedTrait::<FP16x16>::new({val}, {'true' if val < 0 else 'false'})" for val in values]
    return ",\n ".join(values)

input_cairo = generate_input_cairo(test_images_pt_downsampled[0])

with open(output_folder + "input.cairo", "w") as f:
    f.write("""
use array::{SpanTrait, ArrayTrait};
use orion::operators::tensor::{TensorTrait, FP16x16Tensor, Tensor};
use orion::numbers::{FixedTrait, FP16x16};
fn input() -> Tensor<FP16x16> {
    TensorTrait::<FP16x16>::new(
        array![196].span(),
        array![
    """)
    f.write(input_cairo)
    f.write("""
        ].span()
    )
}
    """)

### Build Cairo

In [10]:
command = f'cd {output_folder}/inference/ && scarb build'
subprocess.run(command, shell=True)

    Updating git repository https://github.com/gizatechxyz/orion
    Updating git repository https://github.com/keep-starknet-strange/alexandria
    Updating git repository https://github.com/influenceth/cubit
   Compiling inference v0.1.0 (/home/guy1m0/Desktop/ZKML-Benchmark/frameworks/orion/tmp/196_25_10/inference/Scarb.toml)
warn: Unused variable. Consider ignoring by prefixing with `_`.
 --> /home/guy1m0/.cache/scarb/registry/git/checkouts/orion-m80ng8gqktahi/8ff73895/src/operators/tensor/core.cairo:5439:38
                        Option::Some(item) => {
                                     ^**^

warn: Unused variable. Consider ignoring by prefixing with `_`.
 --> /home/guy1m0/.cache/scarb/registry/git/checkouts/orion-m80ng8gqktahi/8ff73895/src/operators/tensor/core.cairo:5499:42
                            Option::Some(val) => {
                                         ^*^

warn: Unused variable. Consider ignoring by prefixing with `_`.
 --> /home/guy1m0/.cache/scarb/registry/git/

CompletedProcess(args='cd ./tmp/196_25_10//inference/ && scarb build', returncode=0)

In [43]:
input_json = output_folder + "inference/target/dev/inference.sierra.json"
output_json = output_folder + "output.casm.json"

command = ['./starknet-sierra-compile', '--', input_json, output_json]

In [44]:
subprocess.run(command)

Error: deserialization Failed.

Caused by:
    missing field `sierra_program` at line 1 column 8660425


CompletedProcess(args=['./starknet-sierra-compile', '--', './tmp/196_25_10/inference/target/dev/inference.sierra.json', './tmp/196_25_10/output.casm.json'], returncode=1)